In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [0]:
import math
from torch.autograd import Variable

class Embedder(nn.Module):
    def __init__(self, vocab_size, d_model):
        super().__init__()
        self.d_model = d_model
        self.embed = nn.Embedding(vocab_size, d_model)
    def forward(self, x):
        return self.embed(x)

class PositionalEncoder(nn.Module):
    def __init__(self, d_model, max_seq_len = 200, dropout = 0.1):
        super().__init__()
        self.d_model = d_model
        self.dropout = nn.Dropout(dropout)
        # create constant 'pe' matrix with values dependant on 
        # pos and i
        pe = torch.zeros(max_seq_len, d_model)
        for pos in range(max_seq_len):
            for i in range(0, d_model, 2):
                pe[pos, i] = \
                math.sin(pos / (10000 ** ((2 * i)/d_model)))
                pe[pos, i + 1] = \
                math.cos(pos / (10000 ** ((2 * (i + 1))/d_model)))
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
 
    
    def forward(self, x):
        # make embeddings relatively larger
        x = x * math.sqrt(self.d_model)
        #add constant to embedding
        seq_len = x.size(1)
        pe = Variable(self.pe[:,:seq_len], requires_grad=False)
        if x.is_cuda:
            pe.cuda()
        x = x + pe
        return self.dropout(x)

In [0]:
class Norm(nn.Module):
    def __init__(self, d_model, eps = 1e-6):
        super().__init__()
    
        self.size = d_model
        
        # create two learnable parameters to calibrate normalisation
        self.alpha = nn.Parameter(torch.ones(self.size))
        self.bias = nn.Parameter(torch.zeros(self.size))
        
        self.eps = eps
    
    def forward(self, x):
        norm = self.alpha * (x - x.mean(dim=-1, keepdim=True)) \
        / (x.std(dim=-1, keepdim=True) + self.eps) + self.bias
        return norm

def attention(q, k, v, d_k, mask=None, dropout=None):
    
    scores = torch.matmul(q, k.transpose(-2, -1)) /  math.sqrt(d_k)
    
    if mask is not None:
        mask = mask.unsqueeze(1)
        scores = scores.masked_fill(mask == 0, -1e9)
    
    scores = F.softmax(scores, dim=-1)
    
    if dropout is not None:
        scores = dropout(scores)
        
    output = torch.matmul(scores, v)
    return output

class MultiHeadAttention(nn.Module):
    def __init__(self, heads, d_model, dropout = 0.1):
        super().__init__()
        
        self.d_model = d_model #分散表現の次元数
        self.d_k = d_model // heads #headで割った後の次元数
        self.h = heads #ヘッドの数
        
        self.q_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        
        self.dropout = nn.Dropout(dropout)
        self.out = nn.Linear(d_model, d_model)
    
    def forward(self, q, k, v, mask=None):
        
        bs = q.size(0)
        
        # perform linear operation and split into N heads
        k = self.k_linear(k).view(bs, -1, self.h, self.d_k)
        q = self.q_linear(q).view(bs, -1, self.h, self.d_k)
        v = self.v_linear(v).view(bs, -1, self.h, self.d_k)
        
        # transpose to get dimensions bs * N * sl * d_model
        k = k.transpose(1,2)
        q = q.transpose(1,2)
        v = v.transpose(1,2)
        

        # calculate attention using function we will define next
        scores = attention(q, k, v, self.d_k, mask, self.dropout)
        # concatenate heads and put through final linear layer
        concat = scores.transpose(1,2).contiguous().view(bs, -1, self.d_model)
        output = self.out(concat)
    
        return output

class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff=2048, dropout = 0.1):
        super().__init__() 
    
        # We set d_ff as a default to 2048
        self.linear_1 = nn.Linear(d_model, d_ff)
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(d_ff, d_model)
    
    def forward(self, x):
        x = self.dropout(F.relu(self.linear_1(x)))
        x = self.linear_2(x)
        return x

In [0]:
#エンコーダ自体の宣言
class EncoderLayer(nn.Module):
    def __init__(self, d_model, heads, dropout=0.1):
        super().__init__()
        self.norm_1 = Norm(d_model)
        self.norm_2 = Norm(d_model)
        self.attn = MultiHeadAttention(heads, d_model, dropout=dropout)
        self.ff = FeedForward(d_model, dropout=dropout)
        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)
        
    def forward(self, x, mask):
        x2 = self.norm_1(x)
        x = x + self.dropout_1(self.attn(x2,x2,x2,mask))
        x2 = self.norm_2(x)
        x = x + self.dropout_2(self.ff(x2))
        return x
    
# build a decoder layer with two multi-head attention layers and
# one feed-forward layer
# デコーダ自体の宣言
class DecoderLayer(nn.Module):
    def __init__(self, d_model, heads, dropout=0.1):
        super().__init__()
        self.norm_1 = Norm(d_model)
        self.norm_2 = Norm(d_model)
        self.norm_3 = Norm(d_model)
        
        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)
        self.dropout_3 = nn.Dropout(dropout)
        
        self.attn_1 = MultiHeadAttention(heads, d_model, dropout=dropout)
        self.attn_2 = MultiHeadAttention(heads, d_model, dropout=dropout)
        self.ff = FeedForward(d_model, dropout=dropout)

    def forward(self, x, e_outputs, src_mask, trg_mask):
        x2 = self.norm_1(x)
        x = x + self.dropout_1(self.attn_1(x2, x2, x2, trg_mask))
        x2 = self.norm_2(x)
        x = x + self.dropout_2(self.attn_2(x2, e_outputs, e_outputs, \
        src_mask))
        x2 = self.norm_3(x)
        x = x + self.dropout_3(self.ff(x2))
        return x

In [0]:
import copy

def get_clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)])

class Encoder(nn.Module):
    def __init__(self, vocab_size, d_model, N, heads, dropout):
        super().__init__()
        self.N = N
        self.embed = Embedder(vocab_size, d_model)
        self.pe = PositionalEncoder(d_model, dropout=dropout)
        self.layers = get_clones(EncoderLayer(d_model, heads, dropout), N)
        self.norm = Norm(d_model)
    def forward(self, src, mask):
        x = self.embed(src)
        x = self.pe(x)
        for i in range(self.N):
            x = self.layers[i](x, mask)
        return self.norm(x)
    
class Decoder(nn.Module):
    def __init__(self, vocab_size, d_model, N, heads, dropout):
        super().__init__()
        self.N = N
        self.embed = Embedder(vocab_size, d_model)
        self.pe = PositionalEncoder(d_model, dropout=dropout)
        self.layers = get_clones(DecoderLayer(d_model, heads, dropout), N)
        self.norm = Norm(d_model)
    def forward(self, trg, e_outputs, src_mask, trg_mask):
        x = self.embed(trg)
        x = self.pe(x)
        for i in range(self.N):
            x = self.layers[i](x, e_outputs, src_mask, trg_mask)
        return self.norm(x)

class Transformer(nn.Module):
    def __init__(self, src_vocab, trg_vocab, d_model, N, heads, dropout):
        super().__init__()
        self.encoder = Encoder(src_vocab, d_model, N, heads, dropout)
        self.decoder = Decoder(trg_vocab, d_model, N, heads, dropout)
        self.out = nn.Linear(d_model, trg_vocab)
    def forward(self, src, trg, src_mask, trg_mask):
        e_outputs = self.encoder(src, src_mask)
        d_output = self.decoder(trg, e_outputs, src_mask, trg_mask)
        output = self.out(d_output)
        return output

def get_model(src_vocab, trg_vocab, d_model, n_layers, heads, dropout):
    
    #assert opt.d_model % opt.heads == 0
    #assert opt.dropout < 1

    model = Transformer(src_vocab, trg_vocab, d_model, n_layers, heads, dropout)

    """   
    if opt.load_weights is not None:
        print("loading pretrained weights...")
        model.load_state_dict(torch.load(f'{opt.load_weights}/model_weights'))
    else:
        for p in model.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p) 
    """
    for p in model.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p) 
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    model.to(device)

    """
    if opt.device == 0:
        model = model.cuda()
    """

    return model

In [6]:
train_path = 'drive/My Drive/dataset/eng2fre_2/train.txt'
train_data = open(train_path).read().strip().split('\n')
val_path = 'drive/My Drive/dataset/eng2fre_2/val.txt'
val_data = open(val_path).read().strip().split('\n')
"""
input_path = 'drive/My Drive/dataset/eng2fre_2/english.txt'
src = open(input_path).read().strip().split('\n')
output_path = 'drive/My Drive/dataset/eng2fre_2/french.txt'
trg = open(output_path).read().strip().split('\n')
"""

"\ninput_path = 'drive/My Drive/dataset/eng2fre_2/english.txt'\nsrc = open(input_path).read().strip().split('\n')\noutput_path = 'drive/My Drive/dataset/eng2fre_2/french.txt'\ntrg = open(output_path).read().strip().split('\n')\n"

In [7]:
!python -m spacy download fr_core_news_sm

     |████████████████████████████████| 14.7MB 693kB/s 
  Created wheel for fr-core-news-sm: filename=fr_core_news_sm-2.2.5-cp36-none-any.whl size=14727027 sha256=0516b957b35c35809eb59cdcc5303dded440dfd463bde28d23e589e8d542cf74
  Stored in directory: /tmp/pip-ephem-wheel-cache-5lttrmfh/wheels/46/1b/e6/29b020e3f9420a24c3f463343afe5136aaaf955dbc9e46dfc5
Successfully built fr-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_sm')


In [0]:
import spacy
import re
from torchtext import data
import fr_core_news_sm

#トークナイザを定義
class tokenize(object):
    
    def __init__(self, lang):
        self.nlp = spacy.load(lang)
            
    def tokenizer(self, sentence):
        sentence = re.sub(
        r"[\*\"“”\n\\…\+\-\/\=\(\)‘•:\[\]\|’\!;]", " ", str(sentence))
        sentence = re.sub(r"[ ]+", " ", sentence)
        sentence = re.sub(r"\!+", "!", sentence)
        sentence = re.sub(r"\,+", ",", sentence)
        sentence = re.sub(r"\?+", "?", sentence)
        sentence = sentence.lower()
        return [tok.text for tok in self.nlp.tokenizer(sentence) if tok.text != " "]

class tokenize_fr(object):
    
    def __init__(self, lang):
        self.nlp = fr_core_news_sm.load()
            
    def tokenizer(self, sentence):
        sentence = re.sub(
        r"[\*\"“”\n\\…\+\-\/\=\(\)‘•:\[\]\|’\!;]", " ", str(sentence))
        sentence = re.sub(r"[ ]+", " ", sentence)
        sentence = re.sub(r"\!+", "!", sentence)
        sentence = re.sub(r"\,+", ",", sentence)
        sentence = re.sub(r"\?+", "?", sentence)
        sentence = sentence.lower()
        return [tok.text for tok in self.nlp.tokenizer(sentence) if tok.text != " "]


MAX_LENGTH = 20
t_src = tokenize('en_core_web_sm')
t_trg = tokenize_fr('fr_core_news_sm')
#フランス
TRG = data.Field(lower=True, tokenize=t_trg.tokenizer, batch_first=True, 
                 init_token='<sos>', eos_token='<eos>', fix_length=MAX_LENGTH)
#英語
SRC = data.Field(lower=True, tokenize=t_src.tokenizer, batch_first=True, fix_length=MAX_LENGTH)

In [0]:
import pandas as pd

"""
df_train = pd.read_csv('drive/My Drive/dataset/eng2fre_2/train.csv')
df_val = pd.read_csv('drive/My Drive/dataset/eng2fre_2/val.csv')

raw_data = {'src' : [line for line in src], 'trg': [line for line in trg]}
df = pd.DataFrame(raw_data, columns=["src", "trg"])

#文の長さに制限をかける
mask = (df_train['src'].str.count(' ') < MAX_LENGTH) & (df_train['trg'].str.count(' ') < MAX_LENGTH)
df = df_train.loc[mask]

df.to_csv("translate_transformer_temp.csv", index=False)
"""

data_fields = [('src', SRC), ('trg', TRG)]
train_ds, val_ds = data.TabularDataset.splits(
    path='drive/My Drive/dataset/eng2fre_2/', train='train.csv', validation='val.csv', format='csv', fields=data_fields
)

#train_ds, val_ds = data.TabularDataset('./translate_transformer_temp.csv', format='csv', fields=data_fields)

In [10]:
#ボキャブラリを作成
SRC.build_vocab(train_ds)
TRG.build_vocab(train_ds)
print(TRG.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7fdb7f4658c8>, {'<unk>': 0, '<pad>': 1, '<sos>': 2, '<eos>': 3, '.': 4, 'je': 5, 'de': 6, '?': 7, 'pas': 8, 'vous': 9, 'est': 10, 'il': 11, 'que': 12, 'à': 13, 'ne': 14, 'le': 15, 'la': 16, 'tu': 17, 'a': 18, "j'": 19, 'ce': 20, "l'": 21, "n'": 22, 'un': 23, 'tom': 24, ',': 25, 'ai': 26, 'nous': 27, 'en': 28, "d'": 29, 'une': 30, 'les': 31, 'elle': 32, 'me': 33, 'suis': 34, 'pour': 35, "c'": 36, "qu'": 37, '\u202f': 38, 'faire': 39, 'ça': 40, 'dans': 41, 'plus': 42, 'y': 43, "m'": 44, 'des': 45, "s'": 46, 'moi': 47, 'qui': 48, 'tout': 49, 'veux': 50, 'te': 51, 'être': 52, 'se': 53, 'du': 54, 'avec': 55, 'était': 56, 'fait': 57, 'au': 58, 'êtes': 59, 'mon': 60, 'as': 61, 'si': 62, '\xa0': 63, 'sont': 64, 'et': 65, 'son': 66, 'avez': 67, 'ils': 68, 'lui': 69, 'cette': 70, 'es': 71, 'votre': 72, 'très': 73, 'peux': 74, 'temps': 75, 'sur': 76, 'été': 77, 'dit': 78, 'ma': 79, 'ici': 80, "t'": 81, 'cela': 82, 'pourquoi': 83, 'pense': 84, 'chose':

In [11]:
train_dl = data.Iterator(train_ds, batch_size=64, train=True)
val_dl = data.Iterator(val_ds, batch_size=64, train=False, sort=False)
batch = next(iter(val_dl))
print(batch.src.shape)
print(batch.trg.shape)

torch.Size([64, 20])
torch.Size([64, 20])


In [0]:
from torch.autograd import Variable

#デコーダ用に三角形のマスクを作成
def nopeak_mask(size):
    np_mask = np.triu(np.ones((1, size, size)),
    k=1).astype('uint8')
    np_mask =  Variable(torch.from_numpy(np_mask) == 0)
    #if opt.device == 0:
    #  np_mask = np_mask.cuda()
    return np_mask

def create_masks(src, trg):
    #エンコーダ用のpaddingにマスク
    src_mask = (src != SRC.vocab.stoi['<pad>']).unsqueeze(-2)

    #デコーダ用のマスクを作成
    if trg is not None:
        trg_mask = (trg != TRG.vocab.stoi['<pad>']).unsqueeze(-2)
        size = trg.size(1) # get seq_len for matrix
        np_mask = nopeak_mask(size)
        if trg.is_cuda:
            np_mask.cuda()
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        np_mask = np_mask.to(device)
        trg_mask = trg_mask & np_mask #triu、pad両方のmaskでTureのものだけ生き残る
        
    else:
        trg_mask = None
    return src_mask, trg_mask

In [0]:
# ミニバッチの用意
import numpy as np
import torch.optim as optim

d_model = 300 #分散表現の次元数
nx = 3 #Nx
head_num = 5 #headの数
dropout_rate = 0.1

dataloaders_dict = {"train": train_dl, "val": val_dl}
model = get_model(len(SRC.vocab.stoi), len(TRG.vocab.stoi), d_model, nx, head_num, dropout_rate)

#損失関数を定義
criterion = nn.CrossEntropyLoss(ignore_index=TRG.vocab.stoi['<pad>'])

#最適化手法
learning_rate = 2e-5
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [14]:
SRC.vocab.stoi

defaultdict(<function torchtext.vocab._default_unk_index>,
            {'<unk>': 0,
             '<pad>': 1,
             '.': 2,
             'i': 3,
             'you': 4,
             'to': 5,
             'the': 6,
             '?': 7,
             'a': 8,
             "n't": 9,
             'do': 10,
             'is': 11,
             'it': 12,
             'he': 13,
             'that': 14,
             'tom': 15,
             "'s": 16,
             'of': 17,
             'have': 18,
             'me': 19,
             'in': 20,
             'this': 21,
             'was': 22,
             ',': 23,
             'we': 24,
             'what': 25,
             'are': 26,
             'my': 27,
             'for': 28,
             'your': 29,
             'did': 30,
             'she': 31,
             "'m": 32,
             'be': 33,
             'not': 34,
             'want': 35,
             "'re": 36,
             'like': 37,
             'on': 38,
             'with': 39,
   

In [15]:
batch = next(iter(val_dl))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

src = batch.src.to(device)


trg = batch.trg.to(device)
trg_input = trg[:, :-1]
src_mask, trg_mask = create_masks(src, trg_input) #mask部分がFalseに
net1 = get_model(len(SRC.vocab.stoi), len(TRG.vocab.stoi), d_model, nx, head_num, dropout_rate)

init_tok = TRG.vocab.stoi['<sos>']
src_mask = (src != SRC.vocab.stoi['<pad>']).unsqueeze(-2).to(device)
e_output = model.encoder(src, src_mask)

outputs = torch.LongTensor([[init_tok]]).to(device)
    
trg_mask = nopeak_mask(1).to(device)

print(outputs.shape)
print(e_output.shape)
print(src_mask.shape)
print(trg_mask.shape)
"""
out = model.out(model.decoder(outputs, e_output, src_mask, trg_mask))
out = F.softmax(out, dim=-1)

probs, ix = out[:, -1].data.topk(opt.k)
log_scores = torch.Tensor([math.log(prob) for prob in probs.data[0]]).unsqueeze(0)
    
outputs = torch.zeros(opt.k, opt.max_len).long()
if opt.device == 0:
  outputs = outputs.cuda()
outputs[:, 0] = init_tok
outputs[:, 1] = ix[0]
    
e_outputs = torch.zeros(opt.k, e_output.size(-2),e_output.size(-1))
if opt.device == 0:
  e_outputs = e_outputs.cuda()
e_outputs[:, :] = e_output[0]
"""

cuda:0
torch.Size([1, 1])
torch.Size([64, 20, 300])
torch.Size([64, 1, 20])
torch.Size([1, 1, 1])


'\nout = model.out(model.decoder(outputs, e_output, src_mask, trg_mask))\nout = F.softmax(out, dim=-1)\n\nprobs, ix = out[:, -1].data.topk(opt.k)\nlog_scores = torch.Tensor([math.log(prob) for prob in probs.data[0]]).unsqueeze(0)\n    \noutputs = torch.zeros(opt.k, opt.max_len).long()\nif opt.device == 0:\n  outputs = outputs.cuda()\noutputs[:, 0] = init_tok\noutputs[:, 1] = ix[0]\n    \ne_outputs = torch.zeros(opt.k, e_output.size(-2),e_output.size(-1))\nif opt.device == 0:\n  e_outputs = e_outputs.cuda()\ne_outputs[:, :] = e_output[0]\n'

In [0]:
batch = next(iter(val_dl))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
src = batch.src.to(device)
trg = batch.trg.to(device)
trg_input = trg[:, :-1]
src_mask, trg_mask = create_masks(src, trg_input) #mask部分がFalseに

net1 = get_model(len(SRC.vocab.stoi), len(TRG.vocab.stoi), d_model, nx, head_num, dropout_rate)

#self, src, trg, src_mask, trg_mask
preds = net1(src, trg_input, src_mask, trg_mask)

#targetを一つずつずらしたものが正解になる
#[1バッチ目の正解1単語目,1バッチ目の正解2単語目,...,2バッチ目の正解1単語目,...]のように
ys = trg[:, 1:].contiguous().view(-1)
#[バッチ*文長,単語種類]のtensorと正解のインデックスのtensorから損失をだす
loss = criterion(preds.view(-1, preds.size(-1)), ys)
#loss.item()は損失値そのもの

In [17]:
sentence = 'I like dog'
sentence = SRC.preprocess(sentence)
indexed = []
print(sentence)
for tok in sentence:
  if SRC.vocab.stoi[tok] != 0 or opt.floyd == True:
    indexed.append(SRC.vocab.stoi[tok])
  else:
    indexed.append(SRC.vocab.stoi['<unk>'])
    #indexed.append(get_synonym(tok, SRC)) #wordnetによる未知語の補足
#ここでindexedは単語idのリストに
src = Variable(torch.LongTensor([indexed])).to(device)

init_tok = TRG.vocab.stoi['<sos>']
src_mask = (src != SRC.vocab.stoi['<pad>']).unsqueeze(-2)
e_output = model.encoder(src, src_mask)
print(src.shape)
print(src_mask.shape)
print(e_output.shape)


outputs = torch.LongTensor([[init_tok]]).to(device)
    
trg_mask = nopeak_mask(1).to(device)
    
out = model.out(model.decoder(outputs, e_output, src_mask, trg_mask))
out = F.softmax(out, dim=-1)
print(out.shape)

probs, ix = out[:, -1].data.topk(opt.k)
log_scores = torch.Tensor([math.log(prob) for prob in probs.data[0]]).unsqueeze(0)
    
outputs = torch.zeros(opt.k, opt.max_len).long()
if opt.device == 0:
  outputs = outputs.cuda()
outputs[:, 0] = init_tok
outputs[:, 1] = ix[0]
    
e_outputs = torch.zeros(opt.k, e_output.size(-2),e_output.size(-1))
if opt.device == 0:
  e_outputs = e_outputs.cuda()
e_outputs[:, :] = e_output[0]
#return outputs, e_outputs, log_scores


['i', 'like', 'dog']
torch.Size([1, 3])
torch.Size([1, 1, 3])
torch.Size([1, 3, 300])
torch.Size([1, 1, 22535])


NameError: ignored

In [18]:
def translate(model, src, max_len = 80, custom_sentence=False):
    
    model.eval()
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    sentence = SRC.preprocess(src)
    indexed = []
    print(sentence)
    for tok in sentence:
      if SRC.vocab.stoi[tok] != 0 or opt.floyd == True:
        indexed.append(SRC.vocab.stoi[tok])
      else:
        indexed.append(SRC.vocab.stoi['<unk>'])
      #indexed.append(get_synonym(tok, SRC)) #wordnetによる未知語の補足
      #ここでindexedは単語idのリストに
    src = Variable(torch.LongTensor([indexed])).to(device)
    """
    if custom_sentence == True:
      src = tokenize_en(src)
      sentence=Variable(torch.LongTensor([[EN_TEXT.vocab.stoi[tok] for tok in sentence]])).to(device)
    """
    src_mask = (src != SRC.vocab.stoi['<pad>']).unsqueeze(-2)
    e_outputs = model.encoder(src, src_mask)
    
    outputs = torch.zeros(max_len).type_as(src.data)
    outputs[0] = torch.LongTensor([TRG.vocab.stoi['<sos>']])
    for i in range(1, max_len):    
      trg_mask = np.triu(np.ones((1, i, i)).astype('uint8'))
      #trg_mask = nopeak_mask(1).to(device)
      trg_mask = Variable(torch.from_numpy(trg_mask) == 0).to(device)  
      out = model.out(model.decoder(outputs[:i].unsqueeze(0), e_outputs, src_mask, trg_mask))
      out = F.softmax(out, dim=-1) #[1,1,単語数]
      #topのk個をとってくる、valはtopの値でixはそのindex
      #out[:, -1].dataは[[単語数]]
      val, ix = out[:, -1].data.topk(1)
      outputs[i] = ix[0][0] #softmaxが一番大きかったindex
      if ix[0][0] == TRG.vocab.stoi['<eos>']:
        break
    return ' '.join([TRG.vocab.itos[ix] for ix in outputs[:i]])

net1 = get_model(len(SRC.vocab.stoi), len(TRG.vocab.stoi), d_model, nx, head_num, dropout_rate)
sentence = ['How could I forget you?', 'I think I can fix this.',
            'We need some more information.', 'Fasten the gate.', 'He can be relied on.']
max_len = 20
for s in sentence:
  print(translate(net1, s, max_len, True))

['how', 'could', 'i', 'forget', 'you', '?']
<sos> impressionne impressionne impressionne impressionne impressionne impressionne impressionne impressionne impressionne impressionne encombrée encombrée impressionne impressionne impressionne impressionne impressionne impressionne
['i', 'think', 'i', 'can', 'fix', 'this', '.']
<sos> impressionne encombrée encombrée encombrée encombrée encombrée encombrée encombrée encombrée encombrée encombrée encombrée encombrée encombrée encombrée encombrée encombrée encombrée
['we', 'need', 'some', 'more', 'information', '.']
<sos> travailles travailles encombrée encombrée encombrée encombrée encombrée encombrée encombrée encombrée encombrée encombrée encombrée encombrée fatigue impressionne fatigue fatigue
['fasten', 'the', 'gate', '.']
<sos> flickr déjeunez déjeunez déjeunez prémunir prémunir prémunir prémunir prémunir prémunir abrutie abrutie encombrée encombrée encombrée prémunir prémunir prémunir
['he', 'can', 'be', 'relied', 'on', '.']
<sos> trava

In [0]:
np.triu(np.ones((1, 3, 3)).astype('uint8'))

In [0]:
#モデルを訓練して、訓練したモデルをreturnする
#モデル、辞書型で定義したdataloder(イテレータ)、損失関数、オプティマイザ、エポック数を渡す
def train_model(net, dataloaders_dict, optimizer, num_epochs):

    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)
    print('-----start-------')
    # モデルをGPUへ渡す
    net.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    # 各epoch
    for epoch in range(num_epochs):
        # 学習と検証
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # モデルを訓練モードに
            else:
                net.eval()   # モデルを検証モードに

            epoch_loss = 0.0  # epochの損失和
            #epoch_corrects = 0  # epochの正解数

            # 各バッチ
            for batch in (dataloaders_dict[phase]):
                # batchはTextとLableの辞書オブジェクト

                # GPUが使えるならGPUにデータを送る
                src = batch.src.to(device)  # 文章[バッチ, 文長]
                trg = batch.trg.to(device)  # ラベル[バッチ, 文長]
                trg_input = trg[:, :-1] #[バッチ, 文長-1]
                optimizer.zero_grad()
                

                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == 'train'):
                    #マスクを生成する
                    src_mask, trg_mask = create_masks(src, trg_input)
                    #予測
                    preds = model(src, trg_input, src_mask, trg_mask) #[バッチ, 文長-1, 単語の種類]
                    #targetを一つずつずらしたものが正解になる
                    ys = trg[:, 1:].contiguous().view(-1) #[1バッチ目の正解1単語目,1バッチ目の正解2単語目,...,2バッチ目の正解1単語目,...]
                    #損失をだす
                    #loss = criterion(preds.view(-1, preds.size(-1)), ys, ignore_index=opt.trg_pad)
                    #各バッチの平均損失ちを算出
                    loss = F.cross_entropy(preds.view(-1, preds.size(-1)), ys, ignore_index=TRG.vocab.stoi['<pad>'])
                    
                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        #勾配を計算
                        loss.backward()
                        #パラメータの更新
                        optimizer.step()

                    # 各iterの損失を足していく
                    # 各epoch_lossの合計を出すためにわざとバッチサイズをかけている
                    epoch_loss += loss.item() * src.size(0) 
                    
            #1エポック分が終了
            if phase == 'val':
              sentence = ['How could I forget you?', 'I think I can fix this.',
                          'When do you want to eat?', 'What do you think he meant by that?', 'Tom is wiser than me.']
              max_len = 20
              for s in sentence:
                print(translate(net, s, max_len, True))
                
            # epochごとのlossと正解率
            # バッチサイズをかけているので、len(dataloaders_dict[phase].dataset)で割り算して大丈夫
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            print('Epoch {}/{} | {:^5} |  Loss: {:.4f}'.format(epoch+1, num_epochs, phase, epoch_loss))

    return net

In [0]:
"""
#train_model(net, dataloaders_dict, criterion, optimizer, num_epochs)
def train_model(model, dataloaders_dict, criterion, optimizer, num_epochs):

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("training model...")
    model.to(device)
    start = time.time()
    
    for epoch in range(num_epochs):

        total_loss = 0
        #if opt.floyd is False:
        #    print("   %dm: epoch %d [%s]  %d%%  loss = %s" %\
        #    ((time.time() - start)//60, epoch + 1, "".join(' '*20), 0, '...'), end='\r')

        #if opt.checkpoint > 0:
        #    torch.save(model.state_dict(), 'weights/model_weights')

        for i, batch in enumerate(opt.train):
            src = batch.src
            trg = batch.trg
            trg_input = trg[:, :-1]
            #マスクを生成する
            src_mask, trg_mask = create_masks(src, trg_input, opt)
            preds = model(src, trg_input, src_mask, trg_mask)
            #targetを一つずつずらしたものが正解になる
            ys = trg[:, 1:].contiguous().view(-1)
            opt.optimizer.zero_grad()
            loss = F.cross_entropy(preds.view(-1, preds.size(-1)), ys, ignore_index=opt.trg_pad)
            loss.backward()
            opt.optimizer.step()

            #if opt.SGDR == True:
            #    opt.sched.step()

            total_loss += loss.item()

            if (i + 1) % opt.printevery == 0:
                 p = int(100 * (i + 1) / opt.train_len)
                 avg_loss = total_loss/opt.printevery
                 if opt.floyd is False:
                    print("   %dm: epoch %d [%s%s]  %d%%  loss = %.3f" %\
                    ((time.time() - start)//60, epoch + 1, "".join('#'*(p//5)), "".join(' '*(20-(p//5))), p, avg_loss), end='\r')
                 else:
                    print("   %dm: epoch %d [%s%s]  %d%%  loss = %.3f" %\
                    ((time.time() - start)//60, epoch + 1, "".join('#'*(p//5)), "".join(' '*(20-(p//5))), p, avg_loss))
                 total_loss = 0

            #if opt.checkpoint > 0 and ((time.time()-cptime)//60) // opt.checkpoint >= 1:
            #    torch.save(model.state_dict(), 'weights/model_weights')
            #    cptime = time.time()


        print("%dm: epoch %d [%s%s]  %d%%  loss = %.3f\nepoch %d complete, loss = %.03f" %\
        ((time.time() - start)//60, epoch + 1, "".join('#'*(100//5)), "".join(' '*(20-(100//5))), 100, avg_loss, epoch + 1, avg_loss))
"""

In [20]:
len(dataloaders_dict['train'].dataset)

139177

In [23]:
import torch.nn.functional as F 
num_epochs = 10
net_trained = train_model(model, dataloaders_dict, optimizer, num_epochs=num_epochs)

使用デバイス： cuda:0
-----start-------
Epoch 1/10 | train |  Loss: 5.8234
['how', 'could', 'i', 'forget', 'you', '?']
<sos> je je ne vous pas pas ? ?
['i', 'think', 'i', 'can', 'fix', 'this', '.']
<sos> je je ne suis pas pas de la . .
['when', 'do', 'you', 'want', 'to', 'eat', '?']
<sos> tu vous vous que vous vous ? ?
['what', 'do', 'you', 'think', 'he', 'meant', 'by', 'that', '?']
<sos> tu vous que vous vous que tu ne vous pas pas ? ?
['tom', 'is', 'wiser', 'than', 'me', '.']
<sos> tom il est a pas de la . .
Epoch 1/10 |  val  |  Loss: 4.4752
Epoch 2/10 | train |  Loss: 4.0867
['how', 'could', 'i', 'forget', 'you', '?']
<sos> comment comment je vous vous ? ?
['i', 'think', 'i', 'can', 'fix', 'this', '.']
<sos> je je suis suis que je me suis faire être faire . .
['when', 'do', 'you', 'want', 'to', 'eat', '?']
<sos> comment comment vous vous te faire ?
['what', 'do', 'you', 'think', 'he', 'meant', 'by', 'that', '?']
<sos> que que tu as ce que il a fait fait ce que il a dit ?
['tom', 'is', 'wi